In [306]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['patch.force_edgecolor'] = True
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import pgeocode 
from geopy.geocoders import Nominatim # from pypi.org
from geopy.extra.rate_limiter import RateLimiter # from pypi.org
import ssl

In [247]:
data = pd.read_csv("beginner.csv")
data

,Facility Name,Address 1,Address 2,Address 3,City,State,Zip Code,Phone,Fax,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Ohana Memorials Mammography Services,1515 W Yakima Ave.,NaN,NaN,Yakima,WA,98902,509574-3863,509249-5319,NaN,NaN,NaN,NaN
1,31 Medical Group Aviano AB,Italy,31 Medical Group,Unit 6180,NaN,Air Post Office,AE,9604,3.90434E+11,3904343,NaN,NaN,NaN
2,35th Medical Group Misawa AB Japan,Unit 5024,NaN,NaN,APO,AP,96319-5300,8.13118E+11,8117677,NaN,NaN,NaN,NaN
3,374th MDG Unit 5225 SGCR Yokota AB APO AP 96328,Unit 5225 SGCR,374th Medical Group,NaN,APO,AP,96328,425522510,425302026,NaN,NaN,NaN,NaN
4,375th Medical Group,310 W Losey St.,NaN,NaN,Scott AFB,IL,62225-5252,618256-1494,618256-7253,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8806,Zwanger-Pesiri Radiology,160 Brentwood Road,NaN,NaN,Bay Shore,NY,11706,631444-5544,631666-9168,NaN,NaN,NaN,NaN
8807,Zwanger-Pesiri Radiology,803-F Montauk Highway,NaN,NaN,Shirley,NY,11967,631444-5544,631870-8732,NaN,NaN,NaN,NaN
8808,Zwanger-Pesiri Radiology,1729 N Ocean Ave,NaN,NaN,Medford,NY,11763,631444-5544,631870-8732,NaN,NaN,NaN,NaN
8809,Zwanger-Pesiri Radiology Group,759 Montauk Hwy.,NaN,NaN,West Islip,NY,11795,516798-42426001,631870-8732,NaN,NaN,NaN,NaN


In [248]:
cleaned = data[["Facility Name","Address 1","City","State","Zip Code"]]
updated = cleaned.dropna()
updated = updated.reset_index()
updated # deleted unnecessary columns

,index,Facility Name,Address 1,City,State,Zip Code
0,0,Ohana Memorials Mammography Services,1515 W Yakima Ave.,Yakima,WA,98902
1,2,35th Medical Group Misawa AB Japan,Unit 5024,APO,AP,96319-5300
2,3,374th MDG Unit 5225 SGCR Yokota AB APO AP 96328,Unit 5225 SGCR,APO,AP,96328
3,4,375th Medical Group,310 W Losey St.,Scott AFB,IL,62225-5252
4,6,3T Imaging of Morton Grove,9000 Waukengan Rd Ste 110,Morton Grove,IL,60053
...,...,...,...,...,...,...
6948,8805,Zwanger-Pesiri Radiology,987 W. Jericho Turnpike,Smithtown,NY,11787
6949,8806,Zwanger-Pesiri Radiology,160 Brentwood Road,Bay Shore,NY,11706
6950,8807,Zwanger-Pesiri Radiology,803-F Montauk Highway,Shirley,NY,11967
6951,8808,Zwanger-Pesiri Radiology,1729 N Ocean Ave,Medford,NY,11763


In [253]:
i = 0
for index, row in updated.iterrows():
    if row['City'] == 'APO' or row['City'] == 'FPO':
        updated = updated.drop(i)
    i = i + 1

updated #dropped all international locations

,index,Facility Name,Address 1,City,State,Zip Code
0,871,Baylor Scott & White Medical Centers-Capitol Area,2600 E. Pflugerville Pkwy,Pflugerville,TX,TX
2,5705,Petersburg Medical Center,103 Fram St.,Petersburg,AK,99833
3,5244,Norton Sound Health Corporation,1000 Greg Kruschek Ave.,Nome,AK,99762
4,825,Bassett Army Community Hospital,1060 Gaffney Rd.,Ft. Wainwright,AK,99703-7440
5,3222,Imaging Associates,2280 S Woodworth Loop,Palmer,AK,99645
...,...,...,...,...,...,...
6943,7286,Star Valley Health,901 Adams,Afton,WY,83110
6944,4404,Memorial Hospital of Carbon County,2221 W Elm St,Rawlins,WY,82301
6945,5060,North Big Horn Hospital,1115 Lane 12,Lovell,WY,82431
6946,3534,Kaiser Permanente - Northwest,2301 M St NW,MS.RT.R.M,Washington,DC


In [250]:
#updated['Address & Zip'] = updated['Address 1'] + ' ' + updated['Zip Code']
#updated

In [254]:
updated = updated.sort_values('State')
updated # sorted by states

,index,Facility Name,Address 1,City,State,Zip Code
0,871,Baylor Scott & White Medical Centers-Capitol Area,2600 E. Pflugerville Pkwy,Pflugerville,TX,TX
23,331,Anchorage Native Primary Care Clinic,4320 Diplomacy Drive,Anchorage,AK,99508
22,4139,Maniilaq Health Center,436 5th Ave & Ted Stevens Way,Kotzebue,AK,99752
21,332,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,Anchorage,AK,99503
20,4210,Mat-Su Regional Medical Center,2500 S. Woodworth Loop,Palmer,AK,99645
...,...,...,...,...,...,...
6918,6632,Sheridan Memorial Hospital,1401 W 5th St,Sheridan,WY,82801
6917,6285,Rock Creek Medical Imaging,2620 Commercial Way Ste 10,Rock Springs,WY,82901
6931,4408,Memorial Hospital of Sweetwater County,1200 College Drive,Rock Springs,WY,82901
6946,3534,Kaiser Permanente - Northwest,2301 M St NW,MS.RT.R.M,Washington,DC


In [255]:
updated = updated.reset_index(drop = True)
updated

,index,Facility Name,Address 1,City,State,Zip Code
0,871,Baylor Scott & White Medical Centers-Capitol Area,2600 E. Pflugerville Pkwy,Pflugerville,TX,TX
1,331,Anchorage Native Primary Care Clinic,4320 Diplomacy Drive,Anchorage,AK,99508
2,4139,Maniilaq Health Center,436 5th Ave & Ted Stevens Way,Kotzebue,AK,99752
3,332,Anchorage Neighborhood Health Center,4951 Business Park Blvd.,Anchorage,AK,99503
4,4210,Mat-Su Regional Medical Center,2500 S. Woodworth Loop,Palmer,AK,99645
...,...,...,...,...,...,...
6941,6632,Sheridan Memorial Hospital,1401 W 5th St,Sheridan,WY,82801
6942,6285,Rock Creek Medical Imaging,2620 Commercial Way Ste 10,Rock Springs,WY,82901
6943,4408,Memorial Hospital of Sweetwater County,1200 College Drive,Rock Springs,WY,82901
6944,3534,Kaiser Permanente - Northwest,2301 M St NW,MS.RT.R.M,Washington,DC


In [295]:
states = updated.State.value_counts().sort_index().to_frame()
states # converted value counts of each state to dataframe

,State
TX,1
AK,23
AL,104
AR,71
AS,1
...,...
WI,192
WV,54
WY,29
Washington,1


In [302]:
states1 = states.reset_index()
states1.replace({' TX': 'TX'}, regex = True)
states1.sort_values(by=['index'], ascending=True)
states1 # sorted alphabetically

,index,State
0,TX,1
1,AK,23
2,AL,104
3,AR,71
4,AS,1
...,...,...
82,WI,192
83,WV,54
84,WY,29
85,Washington,1


In [303]:
mammogram = pd.read_csv("mammograms.csv")
mammogram # imported a csv we created with location data of all the states & population/location 

,State,Count,Unnamed: 2,Population,Unnamed: 4,Per,Unnamed: 6,Latitude,Longitude
0,Alabama,104,NaN,2531653,NaN,24342.817310,NaN,32.318230,-86.902298
1,Alaska,23,NaN,347065,NaN,15089.782610,NaN,66.160507,-153.369141
2,Arizona,92,NaN,3658425,NaN,39765.489130,NaN,34.048927,-111.093735
3,Arkansas,71,NaN,1535409,NaN,21625.478870,NaN,34.799999,-92.199997
4,California,588,NaN,19843586,NaN,33747.595240,NaN,36.778259,-119.417931
5,Colorado,114,NaN,2851526,NaN,25013.385960,NaN,39.113014,-105.358887
6,Connecticut,89,NaN,1825341,NaN,20509.449440,NaN,41.599998,-72.699997
7,Deleware,20,NaN,502383,NaN,25119.150000,NaN,39.000000,-75.500000
8,Florida,467,NaN,10968797,NaN,23487.788010,NaN,27.994402,-81.760254
9,Georgia,189,NaN,5447400,NaN,28822.222220,NaN,33.247875,-83.441162


In [305]:
mammogram = mammogram[['State','Count','Population','Per','Latitude','Longitude']]
mammogram # delete unnecessary columns

,State,Count,Population,Per,Latitude,Longitude
0,Alabama,104,2531653,24342.817310,32.318230,-86.902298
1,Alaska,23,347065,15089.782610,66.160507,-153.369141
2,Arizona,92,3658425,39765.489130,34.048927,-111.093735
3,Arkansas,71,1535409,21625.478870,34.799999,-92.199997
4,California,588,19843586,33747.595240,36.778259,-119.417931
5,Colorado,114,2851526,25013.385960,39.113014,-105.358887
6,Connecticut,89,1825341,20509.449440,41.599998,-72.699997
7,Deleware,20,502383,25119.150000,39.000000,-75.500000
8,Florida,467,10968797,23487.788010,27.994402,-81.760254
9,Georgia,189,5447400,28822.222220,33.247875,-83.441162
